In [107]:
### Get all Hamburger Country of Origin
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT * WHERE {
    dbr:Hamburger dbo:country ?what

}"""


In [106]:
### What is a hamburger?
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT * WHERE {
    dbr:Hamburger dbo:type ?what

}"""


In [105]:
### Get other main courses
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT * WHERE {
    ?what dbo:type dbr:Main_course
}"""


In [104]:
### Get other main courses
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>

SELECT * WHERE {
    ?what dct:subject dbc:American_cuisine
}"""


In [103]:
### Get other main courses
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>

SELECT * WHERE {
    ?what dct:subject dbc:American_cuisine
}
"""


In [102]:
### Get other main courses
query = """
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>

SELECT * WHERE {
    ?what dct:subject dbc:American_cuisine
}
LIMIT 10000 OFFSET 10000
"""

In [16]:
from etl.dbpedia import DBPediaClient
from db.dbclient import MongoClient

dbpediaclient = DBPediaClient()
ingredients = dbpediaclient.get("""
SELECT * WHERE {
    ?what rdf:type dbo:Food.
    ?what rdf:type owl:Thing.
    ?what rdf:type wikidata:Q2095
}
""")
from_key = lambda x: x['what']['value']
from_url = lambda x: str(x).split('/')[-1]
from_str = lambda x: x.replace("_", " ")
getter = lambda x: from_key(x)

labels = [getter(i) for i in ingredients]

docs = []
for i in labels:
    docs.append({"entity_type": "ingredient", "source": 'dbpedia', 'value': from_url(from_str(i)), 'url': i})
    
mongo_client = MongoClient()
collection = mongo_client.db.get_collection('knowledge')
found_docs = [i for i in collection.find({})]
known_values = [i['value'] for i in found_docs]
for doc in docs:
    if doc['value'] not in known_values:
        collection.insert_one(doc)